<a href="https://colab.research.google.com/github/Goldeno10/flexisaf_Internship_GenAI_DS_Intermediate/blob/main/task_3/task_3_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("http://kaggle.com/datasets/thoughtvector/customer-support-on-twitter")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: goldenoelgenius
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter


100%|██████████| 169M/169M [00:00<00:00, 1.17GB/s]

In [2]:
import re
import nltk
import string
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

full_df = pd.read_csv(
    "customer-support-on-twitter/twcs/twcs.csv",
    nrows=5000
  )

df = full_df[["text"]]
df["text"] = df["text"].astype(str)

# full_df.head()

df.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


# Lower Casing

In [3]:
df["text_lower"] = df["text"].str.lower()
df.tail()

,text,text_lower
4995,"@117290 Hm, they should definitely resume if a...","@117290 hm, they should definitely resume if a..."
4996,"@117290 That's where they need to be on, so ch...","@117290 that's where they need to be on, so ch..."
4997,@115940 Can you guys maybe like....FIX your st...,@115940 can you guys maybe like....fix your st...
4998,@117291 Hulu is only available in the U.S. rig...,@117291 hulu is only available in the u.s. rig...
4999,@115940 when will the platform available in la...,@115940 when will the platform available in la...


# Removal of Punctuations

In [4]:
df.drop(["text_lower"], axis=1,  inplace=True)

PUNCTUATION_TO_REMOVE = string.punctuation

def remove_punctuation(text: str):
  """Removes puntuations text 'text'"""
  return text.translate(
      str.maketrans("", "", PUNCTUATION_TO_REMOVE)
    )

df["text_wo_punct"] = df["text"].apply(
    lambda text: remove_punctuation(text)
  )

df.head()

,text,text_wo_punct
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...
4,@sprintcare I did.,sprintcare I did


# Removal of Stop Words

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

", ".join(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, he'd, he'll, her, here, hers, herself, he's, him, himself, his, how, i, i'd, if, i'll, i'm, in, into, is, isn, isn't, it, it'd, it'll, it's, its, itself, i've, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she'd, she'll, she's, should, shouldn, shouldn't, should've, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, 

In [6]:
STOP_WORDS = set(stopwords.words("english"))

def remove_stopwords(text):
  """Remove stopwords in text 'text'"""
  return " ".join(
      [word for word in str(text).split() if word not in STOP_WORDS]
    )

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))

df.head()


,text,text_wo_punct,text_wo_stop
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...
4,@sprintcare I did.,sprintcare I did,sprintcare I


# Removal of Frequent Words

In [7]:
from collections import Counter

cnt = Counter()

for text in df["text_wo_stop"].values:
  for word in text.split():
    cnt[word] += 1

cnt.most_common(10)

[('I', 1437),
 ('us', 752),
 ('DM', 514),
 ('help', 479),
 ('Please', 376),
 ('We', 338),
 ('Hi', 293),
 ('Thanks', 287),
 ('get', 279),
 ('please', 247)]

In [8]:
FREQ_WORDS = set([w for (w,wc) in cnt.most_common(10)])

def remove_freq_words(text):
  return " ".join(
      [word for word in str(text).split() if word not in FREQ_WORDS]
    )

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(
    lambda text: remove_freq_words(text)
  )

df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare I did,sprintcare I,sprintcare
